In [12]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=e48a64903081daeac952c492d202769052a47c37f45660fa94ca06247da46d4e
  Stored in directory: /Users/liuzhixian/Library/Caches/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [1]:
#-*- coding = utf-8 -*-



import numpy as np
import re   #正则表达，文字匹配
from bs4 import BeautifulSoup  #网页解析
import urllib.request, urllib.response , urllib.error#定制URL， 获取网页数据
import xlsxwriter as xlwt  #进行excel操作
import sqlite3  #进行SQLite数据库操作
import time #time counter
import wget #img download



In [23]:
def main():
    baseurl1 = "https://movie.douban.com/top250?start="



    #Yande.re 图片下载
    baseurl2seg1 = "https://yande.re/post?page="
    baseurl2seg2 = "&tags=fuzichoko"
    datalist = []
    n = int(input("Please enter the page number(int): "))
    for i in range(1,n+1): #调用获取页面信息函数
        #爬取网页
        time_start = time.time() #统计爬取网页时间
        url  = baseurl2seg1 + str(i) + baseurl2seg2
        html = askURL(url)
        #逐一解析网页
        bs = BeautifulSoup(html,"html.parser")
        t_list = bs.select("a[class='directlink largeimg']")
        if t_list == []:
            print("Spider has reached the final page! Total page = %d"%(i-1)) #爬到了最后一页
            break
        datalist.append(t_list)
        time_end = time.time()
        time_spent = (time_end - time_start)
        #输出爬取进度
        print("Page%d success! Time spent = %.2fs"%(i,time_spent))

    print(datalist)
    for item in datalist[0]:
        print(item)
    #分割单一图片URL
    #保存数据
    
    #保存路径
    #savepath= ".\\豆瓣电影top250.xls"
    #保存数据
   # saveData(savepath)

    


def getData(baseurl):
    datalist = []
    #解析数据

    return datalist

 #保存数据
def saveData(savepath):
    print("not yet")
#得到指定一个URL网页信息
def askURL(url):
    #设置用户代理（告诉浏览器我们可以接受什么类型的文件）， 模拟header向服务器发送消息
    head = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15"
    }
    req = urllib.request.Request(url, headers=head)
    html = ""
    try:
        response = urllib.request.urlopen(req,timeout= 10)
        html = response.read().decode("utf-8")
    except urllib.error.URLError as e:
        if hasattr(e,"code"):
            print(e.code)
        if hasattr(e,"reason"):
            print(e.reason)
    return html

In [24]:

#Start
if __name__ == "__main__" :  #当主程序执行时
    main()

Page1 success! Time spent = 3.42s
Page2 success! Time spent = 2.66s
[[<a class="directlink largeimg" href="https://files.yande.re/jpeg/12060ffde5b1da99e31567a564801ec8/yande.re%201064459%20dress%20elise_%28fire_emblem%29%20fire_emblem%20fire_emblem_if%20fuzichoko%20heels%20japanese_clothes%20nintendo%20sakura_%28fire_emblem%29%20torn_clothes%20valentine.jpg"><span class="directlink-info"><img alt="" class="directlink-icon directlink-icon-large" src="https://assets.yande.re/assets/ddl_large-b6a70700b689a23392964bd25e0993d9229c3ffec4c7a4ef5dbe10a516c862d9.gif"/><img alt="" class="directlink-icon directlink-icon-small" src="https://assets.yande.re/assets/ddl-ee9c03fa35a341df36dbc9d247ee4cd820e3534fa21e9f6127fc9dfcd980658e.gif"/><img alt="" class="parent-display" src="https://assets.yande.re/assets/post-star-parent-d285dd41318c1fe1c500726fc01b57e9e600c88f1f2d039eee54015d0e85e499.gif"/><img alt="" class="child-display" src="https://assets.yande.re/assets/post-star-child-e9a0fc526fbf75a32a28

In [ ]:
#urllib test
import urllib.request
import urllib.parse
import urllib.error
#get request
#response = urllib.request.urlopen("https://yande.re/post?tags=yandere")
#print(response.read().decode('utf-8')) #对获取的网页源码进行utf-8解码

#post request 模拟用户真实登陆，包括密码，cookie
#urltest = "https://httpbin.org/post"
#data = bytes(urllib.parse.urlencode({"hello": "world"}),encoding="utf-8")
#response = urllib.request.urlopen(urltest,data= data)
#print(response.read().decode('utf-8'))

#timeout
#try:
#    urltest = "https://httpbin.org/get"
#    response = urllib.request.urlopen(urltest,timeout= 0.01)
#    print(response.read().decode('utf-8'))
#except urllib.error.URLError as e: 
#    print("time out!")

#getheader
#urltest = "https://www.baidu.com"
#response = urllib.request.urlopen(urltest)
#print(response.status)
#print(response.getheaders())
#print(response.getheader("Date"))

#伪装成浏览器
urltest1 = "https://httpbin.org/post"
urltest2 = "http://www.douban.com"
headers = {
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15"
}
#data = bytes(urllib.parse.urlencode({'name':'testor'}),encoding="utf-8")
#req = urllib.request.Request(url=urltest1,data= data,headers= headers,method= "POST")
#response = urllib.request.urlopen(req)
#print(response.read().decode('utf-8'))

req = urllib.request.Request(url=urltest2,headers= headers,method= "GET")
response = urllib.request.urlopen(req)
print(response.read().decode('utf-8'))




<!DOCTYPE HTML>
<html lang="zh-cmn-Hans" class="ua-mac ua-webkit">
<head>
<meta charset="UTF-8">
<meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />
<meta name="description" content="提供图书、电影、音乐唱片的推荐、评论和价格比较，以及城市独特的文化生活。">
<meta name="keywords" content="豆瓣,小组,电影,同城,豆品,广播,登录豆瓣">
<meta property="qc:admins" content="2554215131764752166375" />
<meta property="wb:webmaster" content="375d4a17a4fa24c2" />
<meta name="mobile-agent" content="format=html5; url=https://m.douban.com">
<title>豆瓣</title>
<script>
function set_cookie(t,e,o,n){var i,a,r=new Date;r.setTime(r.getTime()+24*(e||30)*60*60*1e3),i="; expires="+r.toGMTString();for(a in t)document.cookie=a+"="+t[a]+i+"; domain="+(o||"douban.com")+"; path="+(n||"/")}function get_cookie(t){var e,o,n=t+"=",i=document.cookie.split(";");for(e=0;e<i.length;e++){for(o=i[e];" "==o.charAt(0);)o=o.substring(1,o.length);if(0===o.indexOf(n))return o.substring(n.length,o.length).replace(/\"/g,"")}return null}win

BeautifulSoup4 将复杂HTML文档转换成一个复杂的树形结构，每个节点都是python对象，所有对象都可以归纳为四种：
- Tag
- NavigableString
- BeautifulSoup
- Comment

In [78]:
# Test for beautifulsoup
from bs4 import BeautifulSoup
file = open("./Test.html","rb")
html = file.read()

# 用html.parser解析
bs = BeautifulSoup(html,"html.parser")
import re
#print(bs.title)
#print(bs.a)
#print(bs.head)
#print(type(bs.head))
#1.Tag 标签及其内容；拿到它所找的到第一个标签

#print(bs.title.string)
#print(type(bs.title.string))
#2.NavigableString，标签里的内容（字符串）

#print(bs.a.attrs)
#拿标签里的属性

#print(type(bs))
#print(bs.name)
#print(bs.attrs)
#print(bs)
#3.BeautifulSoup 表示整个文档

#print(bs.a.string)
#4.comment 会把注释符号去除

#---------------------------------------------------------

#节点获取，文档的遍历
#print(bs.head.contents)
#i = 7
#print("The %dth: "%i + str(bs.head.contents[i]))
#更多内容见 https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/#id19

#文档的搜索
#find_all 查找所有与字符串匹配的内容
#t_list = bs.find_all("a")
#print(t_list)

#正则表达式搜索：search()
#t_list = bs.find_all(re.compile("a"))
#print(t_list)

#传人一个函数，根据函数的要求来搜索
#def name_exists(tag):
#    return tag.has_attr("name")
#t_list = bs.find_all(name_exists)
#print(t_list)

#keywords 参数
#t_list = bs.find_all(id = "head")
#t_list = bs.find_all(class_ = True) #所有拥有class
#t_list = bs.find_all(class_ = "mnav")

#text 参数
#t_list = bs.find_all(text = "hao123")
#t_list = bs.find_all(text = ["hao123","新闻","更多产品"])
#t_list = bs.find_all(text = re.compile("\d"))

#limit 参数
#t_list = bs.find_all("a",limit=3)
#for item in t_list:
#    print(item)

#css 选择器
#t_list = bs.select('title') #通过标签查找
#t_list = bs.select(".mnav")  #通过类名查找
#t_list = bs.select("#u1") #通过id查找
#t_list = bs.select("a[class='bri']") #通过属性来查找
#t_list = bs.select("head>title") #div>div>div... 通过子标签来查找
t_list = bs.select(".mnav ~ .bri") #通过兄弟标签查找

print(t_list[0].get_text())

#for item in t_list:
#    print(item)

更多产品


In [7]:
#error test
response = urllib.request.urlopen("https://yande.re/post?page=15&tags=fuzichoko")
bs = BeautifulSoup(response,"html.parser")
result = bs.select("a[class='directlink largeimg']")
print(result)

[]
